
*   Computing Platforms: Set up the Workspace for Machine Learning Projects.  https://ms.pubpub.org/pub/computing
*  Machine Learning for Predictions. https://ms.pubpub.org/pub/ml-prediction
* Machine Learning Packages: https://scikit-learn.org/stable/


# Part I: Import and Inspect Data

In [ ]:
# pip install tweepy
!python -m pip install textblob
!python -m textblob.download_corpora


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from textblob import TextBlob

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Rising-Stars-by-Sunshine/stats201-prediction-Colden/main/data/Queried_Data/clipped_trump_data_2021.csv')
df.head()

,Tweet_number,rawContent,timestamp,ID,replyCount,retweetCount,likeCount,item_id
0,0,Excellent idea. Trump can use his foreign emol...,2018-12-31 23:59:45+00:00,1.080000e+18,121,450,1392,2018
1,1,There are truly no limits to the hypocrisy of ...,2018-12-31 23:57:38+00:00,1.080000e+18,143,1593,2640,2018
2,2,Trump lawyer Giuliani Says Assange Should Not ...,2018-12-31 23:49:47+00:00,1.080000e+18,170,1062,2230,2018
3,3,I’d like to take this opportunity to remind ev...,2018-12-31 23:48:46+00:00,1.080000e+18,103,260,1370,2018
4,4,"“Mr Trump, do you think Elizabeth Warren could...",2018-12-31 23:37:22+00:00,1.080000e+18,126,271,1119,2018


# Part II: Prepare the Y varible for Regression

In [ ]:
#test
blob = TextBlob('''Trump says, since Obama has a wall around his house, our border should have a 2,000 mile wall.
There is no wall around Obama's house. That this man can lie so blithely, and we all shrug, is not normal.
He is either mentally ill or a pathological liar or both. Nothing good.''')
pol_and_sub = blob.sentiment
pol = pol_and_sub[0]
sub = pol_and_sub[1]

# create list listpol with values to join
listpol = []
for string in df['rawContent']:
  blob = TextBlob(string)
  pol_and_sub = blob.sentiment
  pol = pol_and_sub[0]
  sub = pol_and_sub[1]
  listpol.append(abs(pol))
  

In [ ]:
# quickly code the function to join the list's values to existing df (I couldn't find a simple pandas documentation to do this)
def add_values_to_dataframe(df, values_list, column_name):
    # create a Series from the list of values
    new_column = pd.Series(values_list)
    # add the new column to the DataFrame
    df[column_name] = new_column
    return df

# call function on df and list to join
df = add_values_to_dataframe(df, listpol, 'Polarization_value')

In [ ]:
df.dtypes
# get average to help inform cutoff point
print(df['Polarization_value'].sum()/77596)

0.013508252761339825


# Part III: Prepare the Y variable for Classification

reference:

https://datatofish.com/if-condition-in-pandas-dataframe/ *italicized text*

In [ ]:
#@title Define Tweet Polarization Parameters
cut_upper = 0.15 #@param {type:"number"}

## 3.1. Method 1: If function

## 3.2. Method 2: Lambda function

notes: the best method that I suggest

In [ ]:
df['Polarization_Bool'] = abs(df['Polarization_value']).apply(lambda x: 1 if x>= cut_upper else 0)
df.head(40)

,Tweet_number,rawContent,timestamp,ID,replyCount,retweetCount,likeCount,item_id,Polarization_value,Polarization_Bool
0,0,Excellent idea. Trump can use his foreign emol...,2018-12-31 23:59:45+00:00,1.080000e+18,121,450,1392,2018,0.437500,1
1,1,There are truly no limits to the hypocrisy of ...,2018-12-31 23:57:38+00:00,1.080000e+18,143,1593,2640,2018,0.084848,0
2,2,Trump lawyer Giuliani Says Assange Should Not ...,2018-12-31 23:49:47+00:00,1.080000e+18,170,1062,2230,2018,0.000000,0
3,3,I’d like to take this opportunity to remind ev...,2018-12-31 23:48:46+00:00,1.080000e+18,103,260,1370,2018,0.320833,1
4,4,"“Mr Trump, do you think Elizabeth Warren could...",2018-12-31 23:37:22+00:00,1.080000e+18,126,271,1119,2018,0.300000,1
5,5,So Graham Norton taking the piss out of Brexit...,2018-12-31 23:33:39+00:00,1.080000e+18,176,415,1220,2018,0.000000,0
6,6,President Trump should be demanding accountabi...,2018-12-31 23:28:58+00:00,1.080000e+18,226,356,1859,2018,0.000000,0
7,7,"Snoop Dogg, Lena Dunham, Barbara Streisand, Ch...",2018-12-31 23:26:38+00:00,1.080000e+18,260,931,3237,2018,0.000000,0
8,8,"Trump says, since Obama has a wall around his ...",2018-12-31 23:21:37+00:00,1.080000e+18,563,4616,15633,2018,0.041667,0
9,9,“Melania Trump Soared as First Lady in 2018 — ...,2018-12-31 22:38:13+00:00,1.080000e+18,430,620,3083,2018,0.712500,1


# Part V Train and Test Split

*reference*:

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
tss = TimeSeriesSplit()
print(tss)

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)


In [ ]:
# change the train and test split parameters 
tss = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None)

In [ ]:
for train_idx, test_idx in tss.split(df):
    print("TRAIN:", train_idx, "TEST:", test_idx)

TRAIN: [   0    1    2 ... 1998 1999 2000] TEST: [2001 2002 2003 ... 3997 3998 3999]
TRAIN: [   0    1    2 ... 3997 3998 3999] TEST: [4000 4001 4002 ... 5996 5997 5998]


In [ ]:
train_idx

array([   0,    1,    2, ..., 3997, 3998, 3999])

In [ ]:
test_idx

array([4000, 4001, 4002, ..., 5996, 5997, 5998])

In [ ]:
train_df = df.filter(items=train_idx, axis=0)
test_df =  df.filter(items=test_idx, axis=0)

In [ ]:
train_df.head()

,Tweet_number,rawContent,timestamp,ID,replyCount,retweetCount,likeCount,item_id,Polarization_value,Polarization_Bool
0,0,Excellent idea. Trump can use his foreign emol...,2018-12-31 23:59:45+00:00,1.080000e+18,121,450,1392,2018,0.437500,1
1,1,There are truly no limits to the hypocrisy of ...,2018-12-31 23:57:38+00:00,1.080000e+18,143,1593,2640,2018,0.084848,0
2,2,Trump lawyer Giuliani Says Assange Should Not ...,2018-12-31 23:49:47+00:00,1.080000e+18,170,1062,2230,2018,0.000000,0
3,3,I’d like to take this opportunity to remind ev...,2018-12-31 23:48:46+00:00,1.080000e+18,103,260,1370,2018,0.320833,1
4,4,"“Mr Trump, do you think Elizabeth Warren could...",2018-12-31 23:37:22+00:00,1.080000e+18,126,271,1119,2018,0.300000,1


In [ ]:
test_df.head()

,Tweet_number,rawContent,timestamp,ID,replyCount,retweetCount,likeCount,item_id,Polarization_value,Polarization_Bool
4000,4000,"""Pence has that veneer of respectability. He's...",2018-12-13 14:38:53+00:00,1.070000e+18,70,465,1354,2018,0.080000,0
4001,4001,’We See You Stephen Miller': White House Repor...,2018-12-13 14:33:41+00:00,1.070000e+18,121,751,1066,2018,0.233333,1
4002,4002,BREAKING NEWS! Trump cancels White House Chri...,2018-12-13 14:32:02+00:00,1.070000e+18,1932,5597,23774,2018,0.200000,1
4003,4003,Trump aides have maintained the McDougal/Danie...,2018-12-13 14:16:43+00:00,1.070000e+18,150,235,1094,2018,0.400000,1
4004,4004,Bob Mueller’s brilliance is that he doesn’t ac...,2018-12-13 14:09:44+00:00,1.070000e+18,978,7541,39349,2018,0.000000,0


# Part VI Prepare the Train and Test Data for Classification and Regression

## 6.1. Classification

### 6.1.1 Define the columns (Y, X) for Classification 

In [ ]:
cols_C = ['Polarization_Bool','retweetCount']

### 6.1.2 Define the Data Frame of Train and Test Data for Classification

In [ ]:
df_C_train = train_df[cols_C]
df_C_test = test_df[cols_C]

### 6.1.3 Export the Train and Test Data for Classification

In [ ]:
df_C_train.head()

,Polarization_Bool,retweetCount
0,1,450
1,0,1593
2,0,1062
3,1,260
4,1,271


In [ ]:
df_C_train.to_csv('Classification_Train.csv')

In [ ]:
df_C_test.head()

,Polarization_Bool,retweetCount
4000,0,465
4001,1,751
4002,1,5597
4003,1,235
4004,0,7541


In [ ]:
df_C_test.to_csv('Classification_Test.csv')

## 6.2 Regression

### 6.2.1. Define the columns (Y, X) for Regression

In [ ]:
cols_R = ['Polarization_value','retweetCount']

### 6.2.2. Define the Data Frame of Train and Test Data for Regression

In [ ]:
df_R_train = train_df[cols_R]
df_R_test = test_df[cols_R]

### 6.2.3. Export the Train and Test Data for Regression

In [ ]:
df_R_train.head()

,Polarization_value,retweetCount
0,0.437500,450
1,0.084848,1593
2,0.000000,1062
3,0.320833,260
4,0.300000,271


In [ ]:
df_R_train.to_csv('Regression_Train.csv')

In [ ]:
df_R_test.head()

,Polarization_value,retweetCount
4000,0.080000,465
4001,0.233333,751
4002,0.200000,5597
4003,0.400000,235
4004,0.000000,7541
4005,0.068182,388
4006,0.075000,897
4007,0.000000,868
4008,0.150000,2855
4009,0.150000,759


In [ ]:
df_R_test.to_csv('Regression_Test.csv')